# 데이터 불어오기

In [1]:
import pandas as pd
df = pd.read_csv('../db/movie_list.csv', index_col=0)
df

,영화ID,영화명,개봉일,국적,장르,감독,배우
0,0,범죄도시2,2022-05-18,한국,"범죄,액션",이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형..."
1,1,아바타: 물의 길,2022-12-14,미국,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛"
2,2,탑건: 매버릭,2022-06-22,미국,액션,조셉 코신스키,"톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그..."
3,3,스파이더맨: 노 웨이 홈,2021-12-15,미국,"액션,어드벤처,SF",존 왓츠,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,제이콥 배덜런,존 파브로,마리사 토메이"
4,4,한산: 용의 출현,2022-07-27,한국,"액션,드라마",김한민,"박해일,이지영,이정태,변요한,안성기,손현주,김성규,김성균,김향기,옥택연,공명,박지환..."
...,...,...,...,...,...,...,...
15839,15839,허수아비들의 땅,2009-04-30,한국,드라마,노경태,"김선영,박원주,김화정,김두홍,신예슬,황상연,박현진,서민정,최경락,최광수,윤심경,임독..."
15840,15840,혹성탈출: 진화의 시작,2011-08-17,미국,"액션,SF,드라마",루퍼트 와이어트,"브라이언 콕스,제임스 프랭코,프리다 핀토"
15841,15841,환상의 빛,2016-07-07,일본,드라마,고레에다 히로카즈,"마키코 에수미,나이토 타카시,아사노 타다노부"
15842,15842,후세: 말하지 못한 내 사랑,2013-03-28,일본,"애니메이션,드라마",미야지 마사유키,"코토부키 미나코,미야노 마모루,코니시 카츠유키"


# 영화 데이터 크롤링

In [2]:
# 패키지 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import numpy as np

In [3]:
# 영화 제목 리스트
movie_list = df['영화명'].values.tolist()
movie_list[:5]

['범죄도시2', '아바타: 물의 길', '탑건: 매버릭', '스파이더맨: 노 웨이 홈', '한산: 용의 출현']

In [8]:
# 크롤링을 통하여 네이버에서 영화 정보를 가져오는 함수
def get_movie_info(movie_list):
  # 페이지 접근
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
  chrome_options.add_argument("headless")
  url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EC%98%81%ED%99%94'
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  driver.implicitly_wait(10)
  driver.get(url)

  # 리스트 생성
  date_list = []
  country_list = []
  genre_list = []
  runtime_list = []
  producer_list = []
  actor_list = []
  summary_list = []
  failed = []

  # 반복문을 통한 크롤링
  for index, movie in enumerate(movie_list):
    print('=' * 50)
    print(f"{index+1}) {movie}")
    print('*' * 50)
    search_box = driver.find_element(By.ID, 'nx_query')
    search_button = driver.find_element(By.XPATH, '//*[@id="nx_search_form"]/fieldset/button')
    search_box.clear()
    search_box.send_keys('영화 ', movie)
    search_button.click()
    time.sleep(0.2)
    try:
      info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[2]/a')
      info.click()
      time.sleep(0.2)
  
    # 한번에 영화검색이 안되면 제목 앞에 '영화' 제거 후 다시 검색
    except:
      try:
        search_box = driver.find_element(By.ID, 'nx_query')
        search_button = driver.find_element(By.XPATH, '//*[@id="nx_search_form"]/fieldset/button')
        search_box.clear()
        search_box.send_keys(movie)
        search_button.click()
        time.sleep(0.2)
        info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[2]/a')
        info.click()    
      except:
        # 기본정보 탭 찾을 수 없는 영화만 프린트 후 다음 영화로 넘어감: 엑셀에서 제목 수정 위한 프로세스
        failed.append(movie)
        print(movie, "실패")
        continue 
      
    # 기본정보 탭 클릭
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    div_date, div_genre, div_country, div_runtime = 0, 0, 0, 0
    info_list = soup.select('div.detail_info > dl > div')
    for j, div in enumerate(info_list):
      div_type = div.select_one('dt').get_text().strip()
      if div_type == "개봉":
        div_date += j+1
      elif div_type == "장르":
        div_genre += j+1
      elif div_type == "국가":
        div_country += j+1
      elif div_type == "러닝타임":
        div_runtime += j+1

    # 개봉일
    try:
      date = soup.select(f'div.detail_info > dl > div:nth-of-type({div_date}) > dd')
      date_list.append((date[0].get_text()))
      print('개봉일 :', date[0].get_text())
    except:
      date_list.append(np.nan)
      print('개봉일 추출 실패')
    # 장르
    try:
      genre = soup.select(f'div.detail_info > dl > div:nth-of-type({div_genre}) > dd')
      genre_list.append((genre[0].get_text().strip(" ")))
      print('장르 :', genre[0].get_text().strip(" "))
    except:
      genre_list.append(np.nan)
      print('장르 추출 실패')
    # 국가
    try:
      country = soup.select(f'div.detail_info > dl > div:nth-of-type({div_country}) > dd')
      country_list.append((country[0].get_text().split(', ')[0]))
      print('국가 :', country[0].get_text().split(', ')[0])
    except:
      country_list.append(np.nan)
      print('장르 추출 실패')
    # 상영시간
    try:
      runtime = soup.select(f'div.detail_info > dl > div:nth-of-type({div_runtime}) > dd')
      runtime_list.append(re.sub(r'[^0-9]', '', runtime[0].get_text()))
      print('상영시간 :', re.sub(r'[^0-9]', '', runtime[0].get_text()))
    except:
      runtime_list.append(np.nan)
      print('상영시간 추출 실패')
    # 줄거리
    try:
      summary = soup.select('div.intro_box > p')
      summary_list.append(summary[0].get_text())
      print('줄거리 성공')
    except:
      summary_list.append(np.nan)
      print('상영시간 추출 실패')

    # 감독/출연 탭 클릭
    producer_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[3]')
    try:
      producer_info.click()
    except:
      producer_info.send_keys(Keys.ENTER)
    time.sleep(0.2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 감독
    try:
      producer = soup.select_one('div.title_box > strong.name > span._text')
      producer_list.append(producer.get_text())
      print('감독 :', producer.get_text())
    except:
      producer_list.append(np.nan)
      print('감독 추출 실패')
    # 배우
    try:
      actors = []
      # 주연
      main_actors = soup.select('div.sec_scroll_cast_member > div:nth-of-type(2) > div > div > div > ul.cast_list > li')
      if main_actors[-1].get_text().strip() == '이어서더보기':
        main_actors_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div[2]/div/div/div/ul/li[11]/a')
        try:
          main_actors_info.click()
        except:
          main_actors_info.send_keys(Keys.ENTER)
        time.sleep(0.2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 페이지 수
        page = int(soup.select_one('div.cm_paging_area > div.pgs > span.npgs > span._total').get_text())
        for i in range(page):
          html = driver.page_source
          soup = BeautifulSoup(html, 'html.parser')
          main_actors = soup.select(f'div#mflick > div > div.list_image_info > ul._panel:nth-of-type({i+1}) > li')
          for i in range(len(main_actors)):
            actor = main_actors[i]
            try:
              actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
            except:
              actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
          if i + 1 < page:
            next_page = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div/a[2]')
            try:
              next_page.click()
            except:
              next_page.send_keys(Keys.ENTER)
        # 뒤로가기
        driver.back()
        time.sleep(0.2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 조연
        if (len(soup.select('div.sec_scroll_cast_member > div')) > 2):
          sub_actors = soup.select('div.sec_scroll_cast_member > div:nth-of-type(3) > div > div > div > ul.cast_list > li')
          if sub_actors[-1].get_text().strip() == '이어서더보기':
            try:
              sub_actors_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div[3]/div/div/div/ul/li[11]/a')
            except:
              sub_actors_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/div/div/ul/li[11]/a')
            try:
              sub_actors_info.click()
            except:
              sub_actors_info.send_keys(Keys.ENTER)
            time.sleep(0.2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # 페이지 수
            page = int(soup.select_one('div.cm_paging_area > div.pgs > span.npgs > span._total').get_text())
            for i in range(page):
              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              sub_actors = soup.select(f'div#mflick > div > div.list_image_info > ul._panel:nth-of-type({i+1}) > li')
              for i in range(len(sub_actors)):
                actor = sub_actors[i]
                try:
                  actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
                except:
                  actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
              if i + 1 < page:
                next_page = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div/a[2]')
                try:
                  next_page.click()
                except:
                  next_page.send_keys(Keys.ENTER)
          else:
            for i in range(len(sub_actors)):
              actor = sub_actors[i]
              try:
                actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
              except:
                actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
      else:
        for i in range(len(main_actors)):
          actor = main_actors[i]
          try:
            actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
          except:
            actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
        # 조연
        if (len(soup.select('div.sec_scroll_cast_member > div')) > 2):
          sub_actors = soup.select('div.sec_scroll_cast_member > div:nth-of-type(3) > div > div > div > ul.cast_list > li')
          if sub_actors[-1].get_text().strip() == '이어서더보기':
            try:
              sub_actors_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div[3]/div/div/div/ul/li[11]/a')
            except:
              sub_actors_info = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/div/div/ul/li[11]/a')
            try:
              sub_actors_info.click()
            except:
              sub_actors_info.send_keys(Keys.ENTER)
            time.sleep(0.2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # 페이지 수
            page = int(soup.select_one('div.cm_paging_area > div.pgs > span.npgs > span._total').get_text())
            for i in range(page):
              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              sub_actors = soup.select(f'div#mflick > div > div.list_image_info > ul._panel:nth-of-type({i+1}) > li')
              for i in range(len(sub_actors)):
                actor = sub_actors[i]
                try:
                  actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
                except:
                  actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
              if i + 1 < page:
                next_page = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div/div[2]/div/a[2]')
                try:
                  next_page.click() 
                except:
                  next_page.send_keys(Keys.ENTER)
          else:
            for i in range(len(sub_actors)):
              actor = sub_actors[i]
              try:
                actors.append(actor.select_one('div.item > div.title_box > strong.name > a').get_text().strip())
              except:
                actors.append(actor.select_one('div.item > div.title_box > strong.name > span').get_text().strip())
      result = ""
      for items in actors:
        if items == actors[-1]:
          result += items
        else:
          result += items + ","
      actor_list.append(result)
      print('배우 :', result)
    except:
      actor_list.append(np.nan)
      print('배우 추출 실패')

  # 드라이버 닫기
  print('종료')
  driver.close()

  # 결과 출력
  print(f"추출 성공한 영화 갯수 : {len(date_list)}")
  print(f"추출 실패한 영화 갯수 : {len(failed)}")
  print(f"추출 실패한 영화 : {failed}")

  return date_list, country_list, genre_list, runtime_list, producer_list, actor_list, summary_list, failed

In [9]:
date_list, country_list, genre_list, runtime_list, producer_list, actor_list, summary_list, failed = get_movie_info(movie_list)

1) 범죄도시2
**************************************************
개봉일 : 2022.05.18.
장르 : 범죄, 액션
국가 : 대한민국
상영시간 : 106
줄거리 성공
감독 : 이상용
배우 : 마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형,이규원,전진오,이다일,김영성,차우진
2) 아바타: 물의 길
**************************************************
개봉일 : 2022.12.14.
장르 : 액션, 모험, SF, 스릴러
국가 : 미국
상영시간 : 192
줄거리 성공
감독 : 제임스 카메론
배우 : 조 샐다나,샘 워싱턴,시고니 위버,우나 채플린,지오바니 리비시,스티븐 랭,케이트 윈슬렛,클리프 커티스,조엘 무어,CCH 파운더,맷 제랄드,제이미 플래터스,브리튼 달튼,베일리 배스,잭 챔피언,필립 겔호,듀언 에반스 주니어,트리니티 블리스
3) 탑건: 매버릭
**************************************************
개봉일 : 2022.06.22.
장르 : 액션
국가 : 미국
상영시간 : 130
줄거리 성공
감독 : 조셉 코신스키
배우 : 톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그렉 타잔 데이비스,찰스 파넬,장 루이자 켈리,대니 라미레즈,발 킬머,모니카 바바로,루이스 풀먼
4) 스파이더맨: 노 웨이 홈
**************************************************
개봉일 : 2021.12.15.
장르 : 액션, 모험, SF
국가 : 미국
상영시간 : 148
줄거리 성공
감독 : 존 왓츠
배우 : 톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,존 파브로,제이콥 배덜런,마리사 토메이,알프리드 몰리나
5) 한산: 용의 출현
**************************************************
개봉일 : 2022.07.27.
장르 : 액션, 드라

In [ ]:
# 데이터 프레임에 저장
movie = pd.DataFrame(df[['영화ID', '영화명']], columns=['영화ID', '영화명'])
movie['개봉일'] = date_list
movie['국가'] = country_list
movie['장르'] = genre_list
movie['상영시간'] = runtime_list
movie['감독'] = producer_list
movie['배우'] = actor_list
movie['줄거리'] = summary_list

In [ ]:
# csv로 저장
movie.to_csv("../db/movie_info.csv", index=False)